In [1]:
import os
import time

import matplotlib.pyplot as plt

import geopandas as gpd
import pandas as pd

from pandarallel import pandarallel
from reaches import readNHD
from utils import specialBuffer

In [32]:
slurm = 197

In [33]:
# FOR NOW, SET
width_set = 'max'

In [34]:
# Control flow
if width_set == 'mean':
    width = 'WidthM'
elif width_set == 'min':
    width = 'WidthM_Min'
elif width_set == 'max':
    width = 'WidthM_Max'
else:
    print('Invalid width option specified, exiting.')
    # sys.exit()

In [35]:
## Prepare data
# Read segmented NHD
segments, huc4, huc2 = readNHD(index=slurm, segmented=True)

NHDPLUS_H_0512_HU4_GDB
read in


In [36]:
pandarallel.initialize()

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [37]:
start = time.time()
segments['buffers'] = segments.parallel_apply(user_defined_function=specialBuffer,
                                                         args=(width, 'flat', True, False),
                                                         axis=1)
end = time.time()

In [43]:
# fig, ax = plt.subplots()
# segments.iloc[0:10].segments.plot(ax=ax, cmap='hsv')
# segments.iloc[0:10].buffers.plot(ax=ax, cmap='summer', alpha=0.7)

In [49]:
# Drop original reach geometry column, set segments as active geometry
segments = segments.drop(columns='segments').set_geometry('buffers').set_crs(crs=segments.crs)

In [52]:
# Write out
# Set write filepath
save_path = '../narrow_rivers_PIXC_data/NHD_buffered/'
save_path = os.path.join(save_path, huc2)
save_file = huc4 + '_buffered.parquet'

#Write out gdf as parquet file
if not os.path.isdir(save_path):
    os.makedirs(save_path)
segments.to_parquet(os.path.join(save_path, save_file))

In [55]:
start = time.time()
test = gpd.read_parquet('/nas/cee-water/cjgleason/fiona/narrow_rivers_PIXC_data/NHD_buffered/HUC2_05/NHDPLUS_H_0512_HU4_GDB_buffered.parquet')
end = time.time()

end - start

35.94800686836243

In [48]:
test

NHDPlusID GNIS_Name  LengthKM    WidthM  WidthM_Min  WidthM_Max  \
0        2.400130e+13      None  0.141586  0.927282    0.558743    1.538905   
0        2.400130e+13      None  0.141586  0.927282    0.558743    1.538905   
0        2.400130e+13      None  0.141586  0.927282    0.558743    1.538905   
0        2.400130e+13      None  0.141586  0.927282    0.558743    1.538905   
0        2.400130e+13      None  0.141586  0.927282    0.558743    1.538905   
...               ...       ...       ...       ...         ...         ...   
1292630  2.400130e+13      None  0.140535  0.395821    0.238506    0.656899   
1292630  2.400130e+13      None  0.140535  0.395821    0.238506    0.656899   
1292630  2.400130e+13      None  0.140535  0.395821    0.238506    0.656899   
1292630  2.400130e+13      None  0.140535  0.395821    0.238506    0.656899   
1292630  2.400130e+13      None  0.140535  0.395821    0.238506    0.656899   

             Bin                                           segments  \
0        (0, 10]  LINESTRING (-9741383.331 4785536.967, -9741401...   
0        (0, 10]  LINESTRING (-9741401.763 4785537.178, -9741407...   
0        (0, 10]  LINESTRING (-9741419.86 4785534.323, -9741420....   
0        (0, 10]  LINESTRING (-9741433.69 4785522.758, -9741434....   
0        (0, 10]  LINESTRING (-9741447.64 4785515.293, -9741453....   
...          ...                                                ...   
1292630  (0, 10]  LINESTRING (-9460959.658 4937327.442, -9460951...   
1292630  (0, 10]  LINESTRING (-9460951.049 4937343.746, -9460947...   
1292630  (0, 10]  LINESTRING (-9460940.424 4937358.669, -9460928...   
1292630  (0, 10]  LINESTRING (-9460928.328 4937372.585, -9460916...   
1292630  (0, 10]  LINESTRING (-9460916.232 4937386.5, -9460904.1...   

                                                   buffers  
0        POLYGON ((-9741401.772 4785536.409, -9741401.7...  
0        POLYGON ((-9741407.069 4785536.47, -9741409.96...  
0        POLYGON ((-9741420.183 4785533.45, -9741425.88...  
0        POLYGON ((-9741433.487 4785521.701, -9741436.2...  
0        POLYGON ((-9741453.739 4785515.887, -9741453.7...  
...                                                    ...  
1292630  POLYGON ((-9460951.34 4937343.9, -9460950.759 ...  
1292630  POLYGON ((-9460947.709 4937350.777, -9460947.6...  
1292630  POLYGON ((-9460928.576 4937372.8, -9460928.08 ...  
1292630  POLYGON ((-9460916.48 4937386.715, -9460915.98...  
1292630  POLYGON ((-9460904.384 4937400.63, -9460903.88...  

[12898820 rows x 9 columns]